In [1]:
%cd ../..

c:\Users\ajaoo\Desktop\Projects\Multivate-forecasting


In [42]:
import os
import time

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

import warnings
from pathlib import Path
from itertools import cycle
import humanize
from darts.metrics import mae, mase, mse
from sklearn.preprocessing import StandardScaler
from src.forecasting.ml_forecasting import (
    FeatureConfig,
    calculate_metrics,
)
from src.utils import plotting_utils
from src.utils.general import LogTime
from src.utils.ts_utils import darts_metrics_adapter, forecast_bias
from tqdm.autonotebook import tqdm
from IPython.display import display, HTML

# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

In [3]:
os.makedirs("reports/figures/ml_forecasting", exist_ok=True)
preprocessed_data_dir = Path("data/processed")
output = Path("reports/figures/ml_forecasting")

In [4]:
def format_plot(
    fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15
):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig


def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample


def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)


def plot_forecast(
    pred_df, forecast_columns, forecast_display_names=None, save_path=None
):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.Set2  # Using a different color palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines",
            marker=dict(size=6, opacity=0.5),
            line=dict(color=act_color, width=2),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=4),
                line=dict(color=next(colors), width=2),
                name=display_col,
            )
        )

    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

In [5]:
data = pd.read_csv(preprocessed_data_dir / "merged_nhs_covid_data.csv", parse_dates=["date"])
data = data.drop(columns=["Unnamed: 0"])
data.head()

,areaCode,areaName,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,latitude,longitude,epi_week
0,E40000007,East of England,2022-09-12,9.0,84162,418.0,47,505.0,2.0,2177579.0,19129.0,6235410,52.24,0.41,202237
1,E40000007,East of England,2022-09-11,8.0,84115,421.0,46,429.0,3.0,2177074.0,19127.0,6235410,52.24,0.41,202237
2,E40000007,East of England,2022-09-10,8.0,84069,419.0,34,296.0,0.0,2176645.0,19124.0,6235410,52.24,0.41,202236
3,E40000007,East of England,2022-09-09,9.0,84035,411.0,34,308.0,2.0,2176349.0,19124.0,6235410,52.24,0.41,202236
4,E40000007,East of England,2022-09-08,9.0,84001,421.0,51,335.0,3.0,2176041.0,19122.0,6235410,52.24,0.41,202236


In [6]:
len(data.areaName.unique())

7

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8054 entries, 0 to 8053
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   areaCode              8054 non-null   object        
 1   areaName              8054 non-null   object        
 2   date                  8054 non-null   datetime64[ns]
 3   covidOccupiedMVBeds   8054 non-null   float64       
 4   cumAdmissions         8054 non-null   int64         
 5   hospitalCases         8054 non-null   float64       
 6   newAdmissions         8054 non-null   int64         
 7   new_confirmed         8054 non-null   float64       
 8   new_deceased          8054 non-null   float64       
 9   cumulative_confirmed  8054 non-null   float64       
 10  cumulative_deceased   8054 non-null   float64       
 11  population            8054 non-null   int64         
 12  latitude              8054 non-null   float64       
 13  longitude         

In [8]:
data = data.sort_values(["areaName", "date"])
data = data.drop(columns=["areaCode", "latitude", "longitude", "population", "epi_week"])
data.head()

,areaName,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased
894,East of England,2020-04-01,0.0,1400,833.0,167,334.0,75.0,2938.0,455.0
893,East of England,2020-04-02,119.0,1584,841.0,184,372.0,71.0,3310.0,526.0
892,East of England,2020-04-03,162.0,1776,914.0,192,350.0,85.0,3660.0,611.0
891,East of England,2020-04-04,171.0,1939,988.0,163,268.0,70.0,3928.0,681.0
890,East of England,2020-04-05,219.0,2159,1230.0,220,281.0,91.0,4209.0,772.0


In [9]:
# # select a single areaName
# area = "East of England"
# data = data[data.areaName == area]
# data = data.drop(columns=["areaName", "areaCode", "latitude", "longitude", "population", "epi_week"])
# data.head()

In [10]:
# from src.transforms.target_transformations import AutoStationaryTransformer
# # Autostatianry 
# transfomer_pipeline = {}
# for _id in tqdm(data["areaName"].unique()):
    
#     auto_stationary = AutoStationaryTransformer(seasonal_period=7)
#     y = data[data["areaName"] == _id]["date","covidOccupiedMVBeds"].set_index("date")
    
#     y_stat = auto_stationary.fit_transform(y, freq="D") 
    
#     data

In [11]:
from src.feature_engineering.autoregressive_features import add_lags

lags = [1, 3, 7, 14, 21, 28]

with LogTime():
    data, added_features = add_lags(
        data, lags=lags, column="covidOccupiedMVBeds", ts_id="areaName", use_32_bit=True
    )
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,covidOccupiedMVBeds_lag_28


In [12]:
# class LogTime:
#     from time import time

#     def __enter__(self):
#         self.start_time = self.time()
#         print("Starting operation...")

#     def __exit__(self, type, value, traceback):
#         elapsed_time = self.time() - self.start_time
#         print(f"Operation completed in {elapsed_time} seconds.")


# def add_lags(df, lags, column):
#     added_features = []
#     for lag in lags:
#         lag_col_name = f"{column}_lag_{lag}"
#         df[lag_col_name] = df[column].shift(lag)
#         added_features.append(lag_col_name)
#     # Drop rows with NaN values
#     df.dropna(inplace=True)
#     return df, added_features


# lags = [1, 7, 14]

# data_with_lags, added_features = add_lags(data, lags, "covidOccupiedMVBeds")

In [13]:
# data = data_with_lags.copy()

## Rolling

In [14]:
from src.feature_engineering.autoregressive_features import add_rolling_features

with LogTime():
    data, added_features = add_rolling_features(
        data,
        rolls=[7, 14, 21],
        column="covidOccupiedMVBeds",
        agg_funcs=["mean", "std"],
        ts_id="areaName",
        use_32_bit=True,
    )
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: covidOccupiedMVBeds_rolling_7_mean,covidOccupiedMVBeds_rolling_7_std,covidOccupiedMVBeds_rolling_14_mean,covidOccupiedMVBeds_rolling_14_std,covidOccupiedMVBeds_rolling_21_mean,covidOccupiedMVBeds_rolling_21_std


In [15]:
from src.feature_engineering.autoregressive_features import add_ewma

with LogTime():
    data, added_features = add_ewma(
        data,
        spans=[7, 14, 21],
        column="covidOccupiedMVBeds",
        ts_id="areaName",
        use_32_bit=True,
    )
print(f"Features Created: {','.join(added_features)}")

Time Elapsed: 0 microseconds
Features Created: covidOccupiedMVBeds_ewma_span_7,covidOccupiedMVBeds_ewma_span_14,covidOccupiedMVBeds_ewma_span_21


In [16]:
# from src.feature_engineering.temporal_features import add_temporal_features

# with LogTime():
#     data, added_features = add_temporal_features(
#         data,
#         field_name="date",
#         frequency="D",
#         add_elapsed=True,
#         drop=False,
#         use_32_bit=True,
#     )
# print(f"Features Created: {','.join(added_features)}")

In [17]:
# Function to calculate the Vax index based on the given description and assumptions
def calculate_vax_index(df):
    # Constants for population and vaccination
    total_population = 60_000_000
    number_of_age_groups = 5
    vaccine_efficacy_first_dose = [0.89, 0.427, 0.76, 0.854]
    vaccine_efficacy_second_dose = [0.92, 0.86, 0.81, None]
    age_group_probabilities_icu = [
        0.01,
        0.02,
        0.05,
        0.1,
        0.15,
    ]  # Hypothetical ICU need probabilities for different age groups
    monthly_vaccination_rate_increase = 0.05
    # Constants for Vaccination start date
    vaccination_start_date = pd.Timestamp("2021-01-18")

    # Divide population into age groups
    population_per_age_group = total_population / number_of_age_groups

    # Initialize Vax index list
    vax_index_list = []

    # Monthly vaccination rate (starting from 0)
    monthly_vaccination_rate = 0

    # Calculate Vax index for each day
    for index, row in df.iterrows():
        # Initialize Si sum for all age groups
        Si_sum = 0

        # If after vaccination start date, increase monthly vaccination rate
        if row["date"].day == 1 and row["date"] > vaccination_start_date:
            monthly_vaccination_rate += monthly_vaccination_rate_increase

        # Calculate Si and Vax index for each age group
        for i in range(number_of_age_groups):
            # Assume vaccination distribution across age groups (aij: two doses, bij: one dose, cij: no doses)
            vaccinated_population = monthly_vaccination_rate * population_per_age_group
            aij = vaccinated_population / 2
            bij = vaccinated_population / 2
            cij = population_per_age_group - aij - bij

            # Calculate S''i based on vaccine efficacy and distribution
            S_double_prime_i = sum(
                wj * aij + vj * (bij - aij)
                for wj, vj in zip(
                    vaccine_efficacy_second_dose, vaccine_efficacy_first_dose
                )
                if wj is not None
            )

            # Calculate Si
            Si = aij + bij + cij - S_double_prime_i

            # Age-specific probability (pi)
            pi = age_group_probabilities_icu[i]

            # Sum Si for all age groups
            Si_sum += pi * (aij + bij + cij - S_double_prime_i) / (aij + bij + cij)

        # Calculate Vax index for the day
        vax_index = Si_sum
        vax_index_list.append(vax_index)

    # Add Vax index to the dataframe
    df["Vax_index"] = vax_index_list
    return df


# Calculating Vax index for the each areaname in the data
data = data.groupby("areaName").progress_apply(calculate_vax_index)

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
# the second wave of the pandemic, the initial data 4 months of 2021, the test data covers september to november 2021
# data = data[(data.date >= "2021-04-01") & (data.date <= "2021-11-30")]

# Find the minimum and maximum dates
min_date = data["date"].min()
max_date = data["date"].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)

# Calculate the date ranges for train, val, and test sets
date_range = max_date - min_date
train_end = min_date + pd.Timedelta(days=date_range.days * 0.75)
val_end = train_end + pd.Timedelta(days=date_range.days * 0.10)

# Split the data into train, validation, and test sets based on the date ranges
train = data[data["date"] < train_end]
val = data[(data["date"] >= train_end) & (data["date"] < val_end)]
test = data[data["date"] >= val_end]

# Calculate the percentage of dates in each dataset
total_samples = len(data)
train_percentage = len(train) / total_samples * 100
val_percentage = len(val) / total_samples * 100
test_percentage = len(test) / total_samples * 100

print(
    f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}"
)
print(
    f"Percentage of Dates in Train: {train_percentage:.2f}% | Percentage of Dates in Validation: {val_percentage:.2f}% | Percentage of Dates in Test: {test_percentage:.2f}%"
)
print(
    f"Max Date in Train: {train.date.max()} | Min Date in Validation: {val.date.min()} | Min Date in Test: {test.date.min()}"
)

Minimum Date: 2020-04-01 00:00:00
Maximum Date: 2022-09-12 00:00:00
# of Training samples: 6039 | # of Validation samples: 801 | # of Test samples: 1214
Percentage of Dates in Train: 74.98% | Percentage of Dates in Validation: 9.95% | Percentage of Dates in Test: 15.07%
Max Date in Train: 2022-01-31 00:00:00 | Min Date in Validation: 2022-02-01 00:00:00 | Min Date in Test: 2022-05-01 00:00:00


In [19]:
# correlation matrix for the features in the data for one area
area = "East of England"
area_data = data[data.areaName == area].drop(columns=["areaName"])
corr = area_data.corr()
fig = px.imshow(corr, x=area_data.columns, y=area_data.columns)
fig.update_layout(
    autosize=False,
    width=900,
    height=900,
    title_text=f"Correlation Matrix for {area}",
    title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
    titlefont={"size": 20},
    legend_title=None,
    legend=dict(
        font=dict(size=15),
        orientation="h",
        yanchor="bottom",
        y=0.98,
        xanchor="right",
        x=1,
    ),
    yaxis=dict(
        title_text="Features",
        titlefont=dict(size=15),
        tickfont=dict(size=15),
    ),
    xaxis=dict(
        title_text="Features",
        titlefont=dict(size=15),
        tickfont=dict(size=15),
    ),
)
fig.show()


## Detecting seasonality

In [20]:
from statsmodels.tsa.stattools import acf

r = acf(data["covidOccupiedMVBeds"])
r = r[1:]  # Remove the autocorrelation at lag 
fig = px.line(x=list(range(1, len(r) + 1)), y=r, title="Autocorrelation of COVID-19 MVBeds")
fig.update_traces(mode="lines+markers", marker=dict(size=4))
fig = format_plot(fig, xlabel="Lag", ylabel="Autocorrelation", title="Autocorrelation of COVID-19 MVBeds")
fig.show()


In [21]:
data.columns

Index(['areaName', 'date', 'covidOccupiedMVBeds', 'cumAdmissions',
       'hospitalCases', 'newAdmissions', 'new_confirmed', 'new_deceased',
       'cumulative_confirmed', 'cumulative_deceased',
       'covidOccupiedMVBeds_lag_1', 'covidOccupiedMVBeds_lag_3',
       'covidOccupiedMVBeds_lag_7', 'covidOccupiedMVBeds_lag_14',
       'covidOccupiedMVBeds_lag_21', 'covidOccupiedMVBeds_lag_28',
       'covidOccupiedMVBeds_rolling_7_mean',
       'covidOccupiedMVBeds_rolling_7_std',
       'covidOccupiedMVBeds_rolling_14_mean',
       'covidOccupiedMVBeds_rolling_14_std',
       'covidOccupiedMVBeds_rolling_21_mean',
       'covidOccupiedMVBeds_rolling_21_std', 'covidOccupiedMVBeds_ewma_span_7',
       'covidOccupiedMVBeds_ewma_span_14', 'covidOccupiedMVBeds_ewma_span_21',
       'Vax_index'],
      dtype='object')

In [22]:
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")

data['day'] = data.index.day
data['month'] = data.index.month

In [23]:
feat_config = FeatureConfig(
    continuous_features=[
        "covidOccupiedMVBeds_lag_1",
        "covidOccupiedMVBeds_lag_3",
        "covidOccupiedMVBeds_lag_7",
        "covidOccupiedMVBeds_lag_14",
        "covidOccupiedMVBeds_lag_21",
        "covidOccupiedMVBeds_lag_28",
        "covidOccupiedMVBeds_roll_7_mean",
        "covidOccupiedMVBeds_roll_7_std",
        "covidOccupiedMVBeds_roll_14_mean",
        "covidOccupiedMVBeds_roll_14_std",
        "covidOccupiedMVBeds_roll_21_mean",
        "covidOccupiedMVBeds_roll_21_std",
        "covidOccupiedMVBeds_ewma_7",
        "covidOccupiedMVBeds_ewma_14",
        "covidOccupiedMVBeds_ewma_21",
        "Vax_index",
    ],
    categorical_features=["day", "month"],
    target="covidOccupiedMVBeds",
    date="date",
)

In [24]:
nc = train.isnull().sum()
nc[nc > 0]

covidOccupiedMVBeds_lag_1                7
covidOccupiedMVBeds_lag_3               21
covidOccupiedMVBeds_lag_7               49
covidOccupiedMVBeds_lag_14              98
covidOccupiedMVBeds_lag_21             147
covidOccupiedMVBeds_lag_28             196
covidOccupiedMVBeds_rolling_7_mean      49
covidOccupiedMVBeds_rolling_7_std       49
covidOccupiedMVBeds_rolling_14_mean     98
covidOccupiedMVBeds_rolling_14_std      98
covidOccupiedMVBeds_rolling_21_mean    147
covidOccupiedMVBeds_rolling_21_std     147
covidOccupiedMVBeds_ewma_span_7          1
covidOccupiedMVBeds_ewma_span_14         1
covidOccupiedMVBeds_ewma_span_21         1
dtype: int64

In [25]:
nc = val.isnull().sum()
nc[nc > 0]

Series([], dtype: int64)

In [26]:
# Missing_Value_Config = MissingValueConfig(
#     bfill_columns=[
#         "covidOccupiedMVBeds_lag_1",
#         "covidOccupiedMVBeds_lag_3",
#         "covidOccupiedMVBeds_lag_7",
#         "covidOccupiedMVBeds_lag_14",
#         "covidOccupiedMVBeds_lag_21",
#         "covidOccupiedMVBeds_lag_28",
#         "covidOccupiedMVBeds_roll_7_mean",
#         "covidOccupiedMVBeds_roll_7_std",
#         "covidOccupiedMVBeds_roll_14_mean",
#         "covidOccupiedMVBeds_roll_14_std",  
#         "covidOccupiedMVBeds_roll_21_mean",
#         "covidOccupiedMVBeds_roll_21_std",
#         "covidOccupiedMVBeds_ewma_7",
#         "covidOccupiedMVBeds_ewma_14",
#         "covidOccupiedMVBeds_ewma_21",
#         "Vax_index",
        
#     ],
#     ffill_columns=[],
#     zero_fill_columns=[],
# )

In [27]:
area = "East of England"
sample_area_train = train.loc[train.areaName == area, :]
sample_area_val = val.loc[val.areaName == area, :]
sample_area_test = test.loc[test.areaName == area, :]

sample_area_train = sample_area_train.drop(columns=["areaName"])
sample_area_val = sample_area_val.drop(columns=["areaName"])
sample_area_test = sample_area_test.drop(columns=["areaName"])

In [28]:
# solving the missing values in the data
sample_area_train = sample_area_train.ffill()
sample_area_val = sample_area_val.ffill()       
sample_area_test = sample_area_test.ffill()

# zero fill the remaining missing values
sample_area_train = sample_area_train.fillna(0) 
sample_area_val = sample_area_val.fillna(0)
sample_area_test = sample_area_test.fillna(0)


In [29]:
sample_area_train.isnull().sum()    

date                                   0
covidOccupiedMVBeds                    0
cumAdmissions                          0
hospitalCases                          0
newAdmissions                          0
new_confirmed                          0
new_deceased                           0
cumulative_confirmed                   0
cumulative_deceased                    0
covidOccupiedMVBeds_lag_1              0
covidOccupiedMVBeds_lag_3              0
covidOccupiedMVBeds_lag_7              0
covidOccupiedMVBeds_lag_14             0
covidOccupiedMVBeds_lag_21             0
covidOccupiedMVBeds_lag_28             0
covidOccupiedMVBeds_rolling_7_mean     0
covidOccupiedMVBeds_rolling_7_std      0
covidOccupiedMVBeds_rolling_14_mean    0
covidOccupiedMVBeds_rolling_14_std     0
covidOccupiedMVBeds_rolling_21_mean    0
covidOccupiedMVBeds_rolling_21_std     0
covidOccupiedMVBeds_ewma_span_7        0
covidOccupiedMVBeds_ewma_span_14       0
covidOccupiedMVBeds_ewma_span_21       0
Vax_index       

In [37]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

# split into X and y
X = sample_area_train.drop(columns=["covidOccupiedMVBeds", "date"])
y = sample_area_train["covidOccupiedMVBeds"]

# Normalizing the features
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.values.reshape(-1, 1))

# split the data into training and validation sets
tscv = TimeSeriesSplit(n_splits=5)
for train_index, val_index in tscv.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]   
    
    print(f"X_train: {X_train.shape} | X_val: {X_val.shape} | y_train: {y_train.shape} | y_val: {y_val.shape}")


X_train: (116, 23) | X_val: (111, 23) | y_train: (116, 1) | y_val: (111, 1)
X_train: (227, 23) | X_val: (111, 23) | y_train: (227, 1) | y_val: (111, 1)
X_train: (338, 23) | X_val: (111, 23) | y_train: (338, 1) | y_val: (111, 1)
X_train: (449, 23) | X_val: (111, 23) | y_train: (449, 1) | y_val: (111, 1)
X_train: (560, 23) | X_val: (111, 23) | y_train: (560, 1) | y_val: (111, 1)


In [43]:
# pred_df = pd.concat([sample_area_train, sample_area_val, sample_area_test])
metric_record = []

In [46]:
# Train machine learning models for forecasting COVID-19 MVBeds for the selected area
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
y_pred = scaler.inverse_transform(y_pred)
y_val = scaler.inverse_transform(y_val)

# Calculate the metrics for the model
metrics = calculate_metrics(y_val, y_pred, name="Linear Regression")
metrics["Model"] = "Linear Regression"
metric_record.append(metrics)
print(metrics)


{'Algorithm': 'Linear Regression', 'MAE': 578579.6264182222, 'MSE': 345774599932.2645, 'MASE': None, 'Forecast Bias': 99.98364867553772, 'Model': 'Linear Regression'}
